In [52]:
import pandas as pd
import sqlite3


In [26]:
import pandas as pd


In [53]:
orders = pd.read_csv("orders.csv")
orders.head()


,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name
0,1,2508,450,18-02-2023,842.97,New Foods Chinese
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian


In [54]:
users = pd.read_json("users.json")
users.head()



,user_id,name,city,membership
0,1,User_1,Chennai,Regular
1,2,User_2,Pune,Gold
2,3,User_3,Bangalore,Gold
3,4,User_4,Bangalore,Regular
4,5,User_5,Pune,Gold


In [55]:
import sqlite3

# create in-memory database
conn = sqlite3.connect(":memory:")

# execute SQL file
with open("restaurants.sql", "r") as f:
    conn.executescript(f.read())

# read restaurants table into pandas
restaurants = pd.read_sql("SELECT * FROM restaurants", conn)
restaurants.head()



,restaurant_id,restaurant_name,cuisine,rating
0,1,Restaurant_1,Chinese,4.8
1,2,Restaurant_2,Indian,4.1
2,3,Restaurant_3,Mexican,4.3
3,4,Restaurant_4,Chinese,4.1
4,5,Restaurant_5,Chinese,4.8


In [56]:
orders_users = pd.merge(
    orders,
    users,
    on="user_id",
    how="left"
)
orders_users.head()


,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name,name,city,membership
0,1,2508,450,18-02-2023,842.97,New Foods Chinese,User_2508,Hyderabad,Regular
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine,User_2693,Pune,Regular
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi,User_2084,Chennai,Gold
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg,User_319,Bangalore,Gold
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian,User_1064,Pune,Regular


In [57]:
final_df = pd.merge(
    orders_users,
    restaurants,
    on="restaurant_id",
    how="left"
)

final_df.head()



,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name_x,name,city,membership,restaurant_name_y,cuisine,rating
0,1,2508,450,18-02-2023,842.97,New Foods Chinese,User_2508,Hyderabad,Regular,Restaurant_450,Mexican,3.2
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine,User_2693,Pune,Regular,Restaurant_309,Indian,4.5
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi,User_2084,Chennai,Gold,Restaurant_107,Mexican,4.0
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg,User_319,Bangalore,Gold,Restaurant_224,Chinese,4.8
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian,User_1064,Pune,Regular,Restaurant_293,Italian,3.0


In [58]:
final_df.to_csv("final_food_delivery_dataset.csv", index=False)


In [43]:
final_df[final_df["membership"] == "Gold"] \
    .groupby("city")["total_amount"] \
    .sum() \
    .sort_values(ascending=False)


city
Chennai      1080909.79
Pune         1003012.32
Bangalore     994702.59
Hyderabad     896740.19
Name: total_amount, dtype: float64

In [44]:
final_df.groupby("cuisine")["total_amount"].mean() \
    .sort_values(ascending=False)


cuisine
Mexican    808.021344
Italian    799.448578
Indian     798.466011
Chinese    798.389020
Name: total_amount, dtype: float64

In [45]:
user_spend = final_df.groupby("user_id")["total_amount"].sum()

user_spend[user_spend > 1000].count()


np.int64(2544)

In [46]:
def rating_bucket(r):
    if 3.0 <= r <= 3.5:
        return "3.0–3.5"
    elif 3.6 <= r <= 4.0:
        return "3.6–4.0"
    elif 4.1 <= r <= 4.5:
        return "4.1–4.5"
    else:
        return "4.6–5.0"

final_df["rating_range"] = final_df["rating"].apply(rating_bucket)

final_df.groupby("rating_range")["total_amount"].sum() \
    .sort_values(ascending=False)


rating_range
4.6–5.0    2197030.75
3.0–3.5    2136772.70
4.1–4.5    1960326.26
3.6–4.0    1717494.41
Name: total_amount, dtype: float64

In [47]:
final_df[final_df["membership"] == "Gold"] \
    .groupby("city")["total_amount"] \
    .mean() \
    .sort_values(ascending=False)


city
Chennai      808.459080
Hyderabad    806.421034
Bangalore    793.223756
Pune         781.162243
Name: total_amount, dtype: float64

In [48]:
restaurant_count = final_df.groupby("cuisine")["restaurant_id"].nunique()
revenue = final_df.groupby("cuisine")["total_amount"].sum()

pd.concat([restaurant_count, revenue], axis=1) \
    .rename(columns={
        "restaurant_id": "restaurant_count",
        "total_amount": "revenue"
    }) \
    .sort_values("restaurant_count")


,restaurant_count,revenue
cuisine,,
Chinese,120,1930504.65
Indian,126,1971412.58
Italian,126,2024203.80
Mexican,128,2085503.09


In [49]:
gold_orders = final_df[final_df["membership"] == "Gold"].shape[0]
total_orders = final_df.shape[0]

round((gold_orders / total_orders) * 100)


50

In [50]:
final_df.groupby(["membership", "cuisine"])["total_amount"].sum() \
    .sort_values(ascending=False)


membership  cuisine
Regular     Mexican    1072943.30
            Italian    1018424.75
Gold        Mexican    1012559.79
            Italian    1005779.05
Regular     Indian      992100.27
Gold        Indian      979312.31
            Chinese     977713.74
Regular     Chinese     952790.91
Name: total_amount, dtype: float64

In [51]:
final_df["order_date"] = pd.to_datetime(final_df["order_date"])
final_df["quarter"] = final_df["order_date"].dt.to_period("Q")

final_df.groupby("quarter")["total_amount"].sum() \
    .sort_values(ascending=False)


quarter
2023Q3    2037385.10
2023Q4    2018263.66
2023Q1    1993425.14
2023Q2    1945348.72
2024Q1      17201.50
Freq: Q-DEC, Name: total_amount, dtype: float64